In [1]:
!pip install mysql-connector-python

  Using cached mysql_connector_python-9.2.0-cp312-cp312-win_amd64.whl.metadata (6.2 kB)
Using cached mysql_connector_python-9.2.0-cp312-cp312-win_amd64.whl (16.1 MB)


In [2]:
import mysql.connector

# Function to establish a database connection
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",    # Change if you have a different username
        password="",    # Add your MySQL password if required
        database="techfix_db"
    )

# Test database connection
try:
    conn = get_db_connection()
    print("Database connection successful!")
    conn.close()
except mysql.connector.Error as err:
    print(f"Error: {err}")


Database connection successful!


In [3]:
def create_users_table():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INT AUTO_INCREMENT PRIMARY KEY,
            full_name VARCHAR(100) NOT NULL,
            email VARCHAR(100) UNIQUE NOT NULL,
            username VARCHAR(50) UNIQUE NOT NULL,
            password VARCHAR(255) NOT NULL,
            user_type ENUM('Admin', 'Supplier') DEFAULT 'Supplier',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    conn.commit()
    conn.close()
    print("✅ Users table created successfully.")

# Run this function to create the table
create_users_table()


✅ Users table created successfully.


In [4]:
import hashlib

def register_user(full_name, email, username, password, user_type="Supplier"):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Hash the password before storing it
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    
    try:
        cursor.execute("""
            INSERT INTO users (full_name, email, username, password, user_type)
            VALUES (%s, %s, %s, %s, %s)
        """, (full_name, email, username, hashed_password, user_type))
        
        conn.commit()
        print(f"✅ User {username} registered successfully.")
    except mysql.connector.Error as err:
        print(f"❌ Error: {err}")
    finally:
        conn.close()

# Example test registration
register_user("John Doe", "john.doe@example.com", "johndoe", "securepassword123", "Supplier")


✅ User johndoe registered successfully.


In [5]:
def get_all_users():
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    
    cursor.execute("SELECT id, full_name, email, username, user_type, created_at FROM users")
    users = cursor.fetchall()
    
    conn.close()
    
    return users

# Display all users
import pandas as pd
users_list = get_all_users()
df = pd.DataFrame(users_list)

import ace_tools as tools
tools.display_dataframe_to_user(name="Users Table", dataframe=df)


ModuleNotFoundError: No module named 'ace_tools'

In [6]:
import pandas as pd

def get_all_users():
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    
    cursor.execute("SELECT id, full_name, email, username, user_type, created_at FROM users")
    users = cursor.fetchall()
    
    conn.close()
    
    return users

# Display all users
users_list = get_all_users()
df = pd.DataFrame(users_list)

# Display DataFrame properly in Jupyter Notebook
from IPython.display import display
display(df)


,id,full_name,email,username,user_type,created_at
0,1,John Doe,john.doe@example.com,johndoe,Supplier,2025-03-07 14:29:19


In [7]:
def login_user(username, password):
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    
    # Hash the input password for comparison
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    
    cursor.execute("""
        SELECT * FROM users WHERE username = %s AND password = %s
    """, (username, hashed_password))
    
    user = cursor.fetchone()
    conn.close()
    
    if user:
        print(f"✅ Login successful. Welcome, {user['full_name']}!")
        return user
    else:
        print("❌ Invalid username or password.")
        return None

# Example test login
login_user("johndoe", "securepassword123")


✅ Login successful. Welcome, John Doe!


{'id': 1,
 'full_name': 'John Doe',
 'email': 'john.doe@example.com',
 'username': 'johndoe',
 'password': 'dda69783f28fdf6f1c5a83e8400f2472e9300887d1dffffe12a07b92a3d0aa25',
 'user_type': 'Supplier',
 'created_at': datetime.datetime(2025, 3, 7, 14, 29, 19)}